In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchinfo
from PIL import Image

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

In [3]:
train_dataset = datasets.ImageFolder(root='/home/danish/Desktop/ML_DL/plant_disease/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train', transform=transform)
val_dataset = datasets.ImageFolder(root='/home/danish/Desktop/ML_DL/plant_disease/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid', transform=transform)

In [4]:
num_classes = len(train_dataset.classes)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [6]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 56 * 56, 2048),
            nn.ReLU(),
            nn.Linear(2048,512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(num_classes).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
torchinfo.summary(model, input_size=(16,3, 224, 224), device=device.type)

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [16, 38]                  --
├─Sequential: 1-1                        [16, 32, 56, 56]          --
│    └─Conv2d: 2-1                       [16, 16, 224, 224]        432
│    └─BatchNorm2d: 2-2                  [16, 16, 224, 224]        32
│    └─ReLU: 2-3                         [16, 16, 224, 224]        --
│    └─MaxPool2d: 2-4                    [16, 16, 112, 112]        --
│    └─Conv2d: 2-5                       [16, 32, 112, 112]        4,608
│    └─BatchNorm2d: 2-6                  [16, 32, 112, 112]        64
│    └─ReLU: 2-7                         [16, 32, 112, 112]        --
│    └─MaxPool2d: 2-8                    [16, 32, 56, 56]          --
├─Sequential: 1-2                        [16, 38]                  --
│    └─Flatten: 2-9                      [16, 100352]              --
│    └─Linear: 2-10                      [16, 2048]                205,522,944
│ 

In [10]:
def calculate_accuracy(y_pred, y_true):
    _, predicted = torch.max(y_pred, 1)
    correct = (predicted == y_true).sum().item()
    return correct / len(y_true)

In [12]:
for epoch in range(5):
    model.train()
    total_loss = 0
    running_accuracy = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        running_accuracy += calculate_accuracy(outputs, labels)

    print(f"Epoch [{epoch+1}/{1}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {running_accuracy/len(train_loader):.4f}")

torch.save(model.state_dict(), 'my_model.pth')

Epoch [1/1], Loss: 0.5963, Accuracy: 0.8133
Epoch [2/1], Loss: 0.4051, Accuracy: 0.8727
Epoch [3/1], Loss: 0.3015, Accuracy: 0.9042
Epoch [4/1], Loss: 0.2422, Accuracy: 0.9232
Epoch [5/1], Loss: 0.1955, Accuracy: 0.9393


In [13]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

Validation Accuracy: 88.62%
